In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.model_selection
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import defaultdict, Counter
from sklearn.neighbors import NearestNeighbors
from sklearn import datasets, linear_model,cluster
from sklearn.metrics import mean_squared_error, r2_score 

### Raw Data Reading 

In [2]:
raw_df = pd.read_csv("zip_housing.csv")
print("Raw data has: " + str(len(raw_df)) + " rows")

Raw data has: 98078 rows


#### Raw Data Processing: Drop all rows where (beds, baths_full, lot_size, building_size) has na, and fill baths_half with 0

In [3]:
dropped_df = raw_df.dropna(subset=['beds','baths_full']) #,'lot_size','building_size'
# Deleting unreasonable data
# dropped_df= dropped_df[dropped_df['building_size'] >= 100]
# dropped_df= dropped_df[dropped_df['lot_size'] >= 100]
dropped_df[['baths_half']] = dropped_df[['baths_half']].fillna(0)
dropped_df[['garage']] = dropped_df[['garage']].fillna(0)
count = 0
df = pd.DataFrame()
for name, group in dropped_df:
    group[["lot_size"]].fillna(group.median(), inplace = True)
    group[["building_size"]].fillna(group.median(), inplace = True)
    group
    if(count == 0):
        df = group
        count += 1
    else:
        df = pd.concat([df, group])
    print("After: " + str(len(group)))
print("Dropped data has: " + str(len(dropped_df)) + " rows")

Dropped data has: 89200 rows


d:\python3\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Since our dataset is large enough, and we do not have much professional knowledge about property markets, we decided to ignore all rows that contain null values instead of filling in estimate values (mean or median).

## DBSCAN Method

#### Normalizing

In [4]:
scaled_dataset = dropped_df[dropped_df.columns[4:11]]
scaled_dataset = preprocessing.normalize(scaled_dataset)
scaled_dataset = pd.DataFrame(scaled_dataset)
scaled_dataset.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

#### Finding Best DBSCAN (eps and Minpoints)

Minpoints is suggested as 2*dimension which should be 14 in our case

In [ ]:
neighbors = NearestNeighbors(n_neighbors = 14)
neighbors_fit = neighbors.fit(scaled_dataset)
distances, indices = neighbors_fit.kneighbors(scaled_dataset)
distances = np.sort(distances, axis=0)
distances = distances[:,1]

#### Zoom in to find 'elbow' optimization point

In [ ]:
plt.plot(distances)
plt.axis([74000, 80000, 0.0002, 0.0010])

#### Fit DBSCAN Model

In [ ]:
minpts = 14
e = 0.0006
db = cluster.DBSCAN(eps=e,min_samples=minpts, metric='euclidean', 
                    metric_params=None, algorithm='auto', 
                    leaf_size=30, p=None, n_jobs=None)

#### Get Labels: 0 for 'not outliers'; 1 for 'outliers'

In [ ]:
model = db.fit(scaled_dataset)
labels = []
outliers = 0
for i in model.labels_:
    if i == -1:
        labels.append(1)
        outliers += 1
    else:
        labels.append(0)
print("Found total: " + str(outliers) + " outliers.")

#### Combine Outliers and Processed Dataset

In [ ]:
dropped_df['outliers'] = labels
noise_free_df = dropped_df[dropped_df['outliers'] == 0]
print("Noise free data has: " + str(len(noise_free_df)) + " rows")

In [ ]:
print('Highest Price: ' + str(max(noise_free_df['price'])))
print('Largest Lot:   ' + str(max(noise_free_df['lot_size'])))

After all processing before, we still have the noise like (price == 915,000,000, or lot_size == 4,356,000,000) which we do not want in our processed dataset. The reason for this may because of the inner work of DBSCAN, which is unsupervised clustering. This raw dataset may contain enough data for those extrodinary large values to become an individual cluster. 

### End of DBSCAN Method

## IQR (interquantile range) Method

#### Find IQR

In [ ]:
iqr_dataset = dropped_df[dropped_df.columns[4:11]]
Q1 = iqr_dataset.quantile(0.25)
Q3 = iqr_dataset.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
lower = Q1 - 1.5 * IQR
higer = Q3 + 1.5 * IQR

In [ ]:
iqr_dataset_out = iqr_dataset[~((iqr_dataset < (lower)) |(iqr_dataset > (higer))).any(axis=1)]

In [ ]:
iqr_df = raw_df.iloc[list(iqr_dataset_out.index),:]
iqr_df[['baths_half']] = iqr_df[['baths_half']].fillna(0)
iqr_df[['garage']] = iqr_df[['garage']].fillna(0)
print("IQR data has: " + str(len(iqr_df)) + " rows")

### Result Graphs for IQR (Blue is original data; Orange is processed data) 

#### Building size

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].plot(np.sort(dropped_df['building_size']))
axs[1].plot(np.sort(iqr_dataset_out['building_size']), 'tab:orange')

#### Price

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].plot(np.sort(dropped_df['price']))
axs[1].plot(np.sort(iqr_dataset_out['price']), 'tab:orange')

#### IQR method is considering all data as a whole. It is not based on any kind of clustering. Although we eliniminated a lot of extrodinary large values, we also eliniminated some reasonable data only because it is larger than the 75% quantile (price). 

### End of IQR method

### Overall, I think the data processed by IQR method is much more reasonable than the DBSCAN method, but either case need to be further adjusted to fulfill our estimatation.